In [1]:
import sys
sys.path.append('..')
from SCCA_new import *
import os
import pickle
import pandas as pd

In [15]:
## Set dimension of parameters and params of main function
n = 200
d = int(200)
s = 5
lam = 0.9

## generate Sigma_x
p1 = d//2
p2 = d - p1
gamma = 0.8
SigX = np.zeros(shape = (p1, p1))
SigY = np.zeros(shape = (p2, p2))

vx = np.zeros(shape= (p1, 1))
vy = np.zeros(shape= (p2, 1))
def block(p):
    mat = np.empty(shape = (p, p))
    for i in range(p):
        for j in range(p):
            mat[i, j] = gamma ** abs(i - j)
    return mat

size_block1 = [p1//10, p1//5, p1//3, p1//5, 0]
cum_sum_size1 = np.append([0],np.cumsum(size_block1))
size_block1[4] = p1 - cum_sum_size1[4]
cum_sum_size1[5] = p1
size_block2 = [p2//3, p2//5, p2//4, p2//8, 0]
cum_sum_size2 = np.append([0],np.cumsum(size_block2))
size_block2[4] = p2 - cum_sum_size2[4]
cum_sum_size2[5] = p2
for i in range(5):
    SigX[cum_sum_size1[i]: cum_sum_size1[i+1] ,cum_sum_size1[i]: cum_sum_size1[i+1]  ] = block(size_block1[i])
    SigY[cum_sum_size2[i]: cum_sum_size2[i+1] ,cum_sum_size2[i]: cum_sum_size2[i+1] ] = block(size_block2[i])

for i in range(0,5*s,5):
    vx[i] = 1/np.sqrt(10)
    vy[i] = 1/np.sqrt(10)

vx = vx / np.sqrt(vx.T.dot(SigX).dot(vx))
vy = vy / np.sqrt(vy.T.dot(SigY).dot(vy))

SigXY = lam * SigX.dot(vx).dot(vy.T).dot(SigY)
Sigma = np.concatenate((np.concatenate((SigX, SigXY), axis=1),
                        np.concatenate((SigXY.T, SigY), axis=1)), axis = 0)

A = np.vstack((np.hstack((np.zeros(shape = (p1, p1)), SigXY))
                    ,np.hstack((SigXY.T, np.zeros(shape = (p2, p2))))))

B = np.vstack((np.hstack((SigX, np.zeros(shape = (p1, p2))))
                    ,np.hstack((np.zeros(shape = (p2, p1)), SigY)))) / (n*2)

In [16]:
size_block1

[10, 20, 33, 20, 17]

In [17]:
size_block2

[33, 20, 25, 12, 10]

In [18]:
size_block1

[10, 20, 33, 20, 17]

In [19]:
size_block2

[33, 20, 25, 12, 10]

In [21]:
# Generate test dataset
folder = '%i-%i_%i_%.1f_%.1f/' %(p1,p2, n,gamma, lam)
isExist = os.path.exists('data/' + folder)

if not isExist:
  
  # Create a new directory because it does not exist 
    os.makedirs('data/' + folder)
    print(folder)
    print("The new directory is created!")

for nrun in range(100):
    Z =  stats.multivariate_normal.rvs(np.zeros(d), Sigma, n)
    X = Z[:, 0:p1]
    Y = Z[:,p1:]
    # center columns of X

    Sighat_X = (X-np.mean(X)).T.dot(X-np.mean(X))
    Sighat_Y = (Y-np.mean(Y)).T.dot(Y - np.mean(Y))
    Sighat_XY = (X-np.mean(X)).T.dot(Y - np.mean(Y))

    Ahat = np.vstack((np.hstack((np.zeros(shape = (p1, p1)), Sighat_XY))
                        ,np.hstack((Sighat_XY.T, np.zeros(shape = (p2, p2)))))) /n

    Bhat = np.vstack((np.hstack((Sighat_X, np.zeros(shape = (p1, p2))))
                        ,np.hstack((np.zeros(shape = (p2, p1)), Sighat_Y)))) /n
    init =  np.random.normal(size = (d))
    np.savetxt('data/'+ folder+'data%i.csv' %nrun, Z, delimiter = ",")
    np.savetxt('data/'+ folder+'Ahat%i.csv' %nrun, Ahat, delimiter = ",")
    np.savetxt('data/'+ folder+'Bhat%i.csv' %nrun, Bhat, delimiter = ",")
    np.savetxt('data/'+ folder+'init%i.csv' %nrun, init, delimiter = ",")

np.savetxt('data/'+ folder+'SigX.csv', SigX, delimiter = ",")
np.savetxt('data/'+ folder+'SigY.csv', SigY, delimiter = ",")
np.savetxt('data/'+ folder+'vy.csv', vy, delimiter = ",")
np.savetxt('data/'+ folder+'vx.csv', vx, delimiter = ",")